In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import statsmodels.api as sm
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

In [ ]:
data = pd.read_csv("/content/NSSO68main.csv", low_memory=False)

In [ ]:
display (data)

,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,4.100000e+31,1,41000,68,10,1,2,24,242,...,0.00,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2.0,12.000000,154.18
1,2,4.100000e+31,1,41000,68,10,1,2,24,242,...,0.00,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2.0,333.000000,484.95
2,3,4.100000e+31,1,41000,68,10,1,2,24,242,...,0.00,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2.0,35.000000,214.84
3,4,4.100000e+31,1,41000,68,10,1,2,24,242,...,0.00,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2.0,168.333333,302.30
4,5,4.100000e+31,1,41000,68,10,1,2,24,242,...,0.00,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2.0,15.000000,148.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,5502,4.180000e+31,1,41810,68,10,1,2,24,241,...,0.00,0.0,0.0,25.000000,683.450500,22.233187,GUJ,1.0,72.375000,188.85
5502,5503,4.180000e+31,1,41810,68,10,1,2,24,241,...,0.01,0.0,0.0,0.010000,967.512000,38.041215,GUJ,1.0,3.000000,234.00
5503,5504,4.180000e+31,1,41810,68,10,1,2,24,241,...,0.00,0.0,0.0,50.000000,610.844000,20.527926,GUJ,1.0,39.200000,126.20
5504,5505,4.180000e+31,1,41810,68,10,1,2,24,241,...,0.00,0.0,0.0,28.000000,387.158600,15.820340,GUJ,1.0,21.880000,86.28


In [ ]:
print(data.columns)

Index(['slno', 'grp', 'Round_Centre', 'FSU_number', 'Round', 'Schedule_Number',
       'Sample', 'Sector', 'state', 'State_Region',
       ...
       'pickle_v', 'sauce_jam_v', 'Othrprocessed_v', 'Beveragestotal_v',
       'foodtotal_v', 'foodtotal_q', 'state_1', 'Region', 'fruits_df_tt_v',
       'fv_tot'],
      dtype='object', length=384)


In [ ]:
data['non_vegetarian'] = (data[['eggsno_q', 'fishprawn_q', 'goatmeat_q', 'beef_q', 'pork_q', 'chicken_q', 'othrbirds_q']].sum(axis=1) > 0).astype(int)
y = data['non_vegetarian']
X = data[['Age','MPCE_MRP', 'Sex', 'Sector']]

In [ ]:
class TobitModel:
    def __init__(self, endog, exog, lower=None, upper=None):
        self.endog = endog
        self.exog = exog
        self.lower = lower
        self.upper = upper

    def loglik(self, params):
        beta = params[:-1]
        sigma = params[-1]
        mu = np.dot(self.exog, beta)

        # Ensure sigma is positive
        sigma = np.abs(sigma) + 1e-10

        # Calculate the log-likelihood
        llf = np.zeros_like(self.endog, dtype=float)

        # Censored from below
        if self.lower is not None:
            llf = np.where(
                self.endog == self.lower,
                np.log(np.clip(norm.cdf((self.lower - mu) / sigma), 1e-10, 1)),
                llf
            )

        # Censored from above
        if self.upper is not None:
            llf = np.where(
                self.endog == self.upper,
                np.log(np.clip(1 - norm.cdf((self.upper - mu) / sigma), 1e-10, 1)),
                llf
            )

        # Uncensored
        uncensored = (self.endog > self.lower) & (self.endog < self.upper)
        llf[uncensored] = -0.5 * np.log(2 * np.pi) - np.log(sigma) - (self.endog[uncensored] - mu[uncensored]) ** 2 / (2 * sigma ** 2)

        return -np.sum(llf)

    def fit(self):
        start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        res = minimize(self.loglik, start_params, method='L-BFGS-B')
        return res

In [ ]:
y_tobit = np.clip(y, 0, 1)

In [ ]:
X_tobit = sm.add_constant(X)

In [ ]:
model = TobitModel(y_tobit, X_tobit, lower=0, upper=1)
results = model.fit()

In [ ]:
print("Tobit Model Results:", results)

Tobit Model Results:   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 3770.118157097058
        x: [-1.702e+03 -5.863e+01  4.052e+04  1.038e+05  6.506e+05]
      nit: 61
      jac: [ 5.912e-04  3.752e-02  1.820e-04  1.365e-04 -4.542e-05]
     nfev: 618
     njev: 103
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
